In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [6]:
data_date = "2018_04_23_15_33_03"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-2/" + data_date

In [7]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [8]:
params.print()

requestedTRamp 1200.0
configuredTRamp 1200.0
startFrequency 5725000000.0
stopFrequency 5875000000.0
frequencyBand 150000000.0
frequencyDevReq 50000.0
frequencyDevConf 50001.1
numberOfSteps 3000
waveformType 1
rampsPerBuffer 750
bufferSize 36000000.0
buffersPerFile 2
rampsPerFile 1500
fileSize 72000000.0
samplingFrequency 10000000.0
samplesPerRamp 12000
skipNSamples 0


In [9]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 71
#firstFile = 37
#nbFiles = 33
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 1500


# Read the data

In [10]:
scalingFactor = 2 / 65535
offset = -32768

In [11]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )

## Read all files

In [12]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINK( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 0 / 140
k = 1, block = 2 / 140
k = 2, block = 4 / 140
k = 3, block = 6 / 140
k = 4, block = 8 / 140
k = 5, block = 10 / 140
k = 6, block = 12 / 140
k = 7, block = 14 / 140
k = 8, block = 16 / 140
k = 9, block = 18 / 140
k = 10, block = 20 / 140
k = 11, block = 22 / 140
k = 12, block = 24 / 140
k = 13, block = 26 / 140
k = 14, block = 28 / 140
k = 15, block = 30 / 140
k = 16, block = 32 / 140
k = 17, block = 34 / 140
k = 18, block = 36 / 140
k = 19, block = 38 / 140
k = 20, block = 40 / 140
k = 21, block = 42 / 140
k = 22, block = 44 / 140
k = 23, block = 46 / 140
k = 24, block = 48 / 140
k = 25, block = 50 / 140
k = 26, block = 52 / 140
k = 27, block = 54 / 140
k = 28, block = 56 / 140
k = 29, block = 58 / 140
k = 30, block = 60 / 140
k = 31, block = 62 / 140
k = 32, block = 64 / 140
k = 33, block = 66 / 140
k = 34, block = 68 / 140
k = 35, block = 70 / 140
k = 36, block = 72 / 140
k = 37, block = 74 / 140
k = 38, block = 76 / 140
k = 39, block = 78 / 140
k = 40, block =

In [13]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [14]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [15]:
rampDownFirst = 1

## Spectrum

### No window

In [16]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [17]:
ifft = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [ ]:
#ifft_abs = 20 * np.log10( np.abs( ifft ) )

In [ ]:
if 0:
    plt.figure()
    plt.imshow( ifft_abs[::100, :] )
    title = "no window - ifft_abs - " + data_date
    plt.title(title)
    plt.savefig( data_dir + "/" + title + ".png" )

In [18]:
plt.figure()
for k in range(10):
    plt.plot( 20 * np.log10( np.abs( np.fft.fft( ifft[:,k] ) ) ), label="{}".format(k) )
plt.legend()
plt.grid()
plt.title(data_date)

In [20]:
plt.figure()
for k in range(10):
    plt.plot( np.abs( ifft[:,k] ), label="{}".format(k) )
plt.legend()
plt.grid()
plt.title(data_date)

## Build RD

In [ ]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst )

In [ ]:
coupling_RD = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)